In [66]:
from Bio import SeqIO
from Bio import SwissProt
from pprint import pprint
%run settings/db_connect.ipynb

In [67]:
kw_name2ac = sql("select name, ac from keywords").ac.to_dict()
go_mf_set = set( sql("select id from goterms where namespace='MF'").index )

In [68]:
handle = open("./data/IN/uniprot/uniprot_sprot.dat")

In [69]:
sp_iterator = SwissProt.parse(handle)

In [61]:
entry = next(sp_iterator)

In [70]:
def to_sql_array(ls):
    return  "{%s}" % ",".join('"%s"'%x for x in ls)

In [71]:
records = []

for e in sp_iterator:
# for e in [entry]:
    go = [(go, rest[0][0]) for cr_id, go, *rest in e.cross_references if cr_id =="GO"]
    gomf = [go for go, namespace in go if namespace =='F']
    kw = [kw_name2ac[name] for name in e.keywords]
    
    record = \
    ( e.accessions[0]            #ID, zapravo primarni AC
    , e.entry_name               #name, zapravo ID (terminologija...)
    , to_sql_array(e.accessions[1:])           #rest of accsesions

    , e.organism_classification[-1]
    , e.PE
    , to_sql_array(kw)
    , "{%s}" % ", ".join(to_sql_array(x) for x in go)
    , to_sql_array(gomf)
    , e.sequence_length
    , e.sequence
    )
    
    records.append(record)
    
records = pd.DataFrame(records, columns=["id", 'name', 'accessions', 'organism', 'pe', 'kw', 'go', 'gomf', 'seq_length', 'seq']).set_index('id')

In [72]:
records.to_sql("swissprot", db_engine, if_exists='append', chunksize=1000)
# df = pd.read_csv('./data/IN/uniprot/swiss_prot.csv').set_index('id')

In [119]:
# records.to_csv('./data/IN/uniprot/swiss_prot.csv')